In [1]:
import numpy as np 
from matplotlib import pyplot as plt 

plt.rcParams.update({
    "text.usetex": True,            # Use LaTeX for all text
    "font.family": "serif",         # Use serif font
    "font.serif": ["Computer Modern Roman"],  # LaTeX default
    "axes.labelsize": 12,
    "font.size": 12,
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
}) 

import os 
from pathlib import Path 

import re 


In [2]:
FOLDER = "MNIST_STRATIFIED_CLASSIFIERS_MADGAN_NEW"

DATASET, _, _, TYPE, _ = FOLDER.split('_')

metrics = ["val_accuracy", "val_f1_score", "val_loss"]   

base_path = Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/notebooks")
strat_exp_path = base_path / 'experiments' / FOLDER 

experiments = os.listdir(strat_exp_path)


In [3]:
cmap = plt.get_cmap('tab10')  # good for distinct colors (up to 10)
colors = [cmap(i) for i in range(10)] + [(0.5, 0.5, 0.5, 1.0)]  # Add gray for the 11th color

color_dict_by_generator = {str(i): colors[i] for i in range(11)}


def extract_info_from_experiment_name(exp: str) -> dict:
    """
    Extracts metadata (dataset, generator info, image counts) from an experiment name string.
    Handles missing generator information gracefully by assigning default integer values.

    Args:
        exp (str): The experiment name string.

    Returns:
        dict: A dictionary containing the extracted metadata, with values converted to appropriate types.
              Returns an empty dictionary if the name doesn't match the expected pattern.
    """
    # General pattern supporting both BASE and MADGAN variants
    # Made generator parts optional and non-greedy where appropriate
    pattern = (
        r".*?_" # Match anything non-greedily up to the first underscore before dataset
        r"(?P<dataset>[A-Z0-9]+)_+" # Dataset name (allow numbers like CIFAR10)
        # Optional group for generator info - make the whole thing optional
        r"(?:" # Start non-capturing group for optional generator parts
            # Optional used_generator part (non-greedy match before it)
            r"(?:.*?" # Non-greedy match any char
              r"used_generator_(?P<used_gen>\d+)_+" # Capture used_gen
            r")?" # End optional used_generator part
            # Optional trained_generators part (non-greedy match before it)
            r"(?:.*?"
              r"trained_generators_(?P<n_gen>\d+)_+" # Capture n_gen
            r")?" # End optional trained_generators part
        r")?" # End non-capturing group for optional generator parts
        # Mandatory image count part (allow non-greedy match before it)
        r".*?"
        r"images_real_(?P<n_real>\d+)_+" # Capture n_real
        r"gen_(?P<n_fake>\d+)" # Capture n_fake (allow pattern to end here or have more chars)
        r".*$" # Match any remaining characters to the end of the string
    )


    match = re.search(pattern, exp)
    if not match:
        print(f"ALARM: Could not parse experiment name: '{exp}'")
        # You might want to return defaults or raise an error depending on requirements
        return {'dataset': 'UNKNOWN', 'used_gen': 0, 'n_gen': 0, 'n_real': 0, 'n_fake': 0} # Example defaults

    d = match.groupdict()

    # Safely convert captured groups to integers, providing a default value (e.g., 0)
    # if the optional groups (used_gen, n_gen) were not matched (resulting in None).
    # The 'or 0' idiom handles None gracefully for int conversion.
    d['used_gen'] = int(d['used_gen'] or 0)
    d['n_gen'] = int(d['n_gen'] or 0)

    # n_real and n_fake should ideally always be matched if the overall pattern matches,
    # but add checks/defaults for robustness in case the regex fails unexpectedly.
    d['n_real'] = int(d['n_real'] or 0)
    d['n_fake'] = int(d['n_fake'] or 0)

    # Note: The keys like 'dataset' remain strings. Only numeric parts are converted.

    return d

def sort_dict_based_on_n_real_images(d: dict, reverse=True) -> dict: 
    return dict(sorted(
        d.items(),
        key=lambda x: int(re.search(r'images_real_(\d+)', x[0]).group(1)), 
        reverse=reverse
    ))



In [4]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import warnings

def plot_history_strat_classifiers(histories: dict, meta_info: dict, save_path: Path, target_gen, METRIC, title_prefix, title_suffix, show: bool = False) -> None:
    """
    Plots the history of a given metric across multiple experiments.
    Dynamically identifies baseline runs (n_real=5000, n_fake=0).
    If multiple baselines are found, selects one, plots it, and removes the others
    from the data structures used for plotting (operates on copies).
    Highlights best/worst non-baseline runs and shows average/median.

    Args:
        histories (dict): Dictionary of history objects.
        meta_info (dict): Dictionary of metadata for experiments.
                          MUST contain 'n_real' and 'n_fake' keys.
        save_path (Path): Path object where the plot image should be saved.
        target_gen (str or int): The total number of generators (used in title).
        METRIC (str): The metric key to plot (e.g., "val_accuracy").
        show (bool, optional): If True, display the plot. Defaults to False.

    Note:
        This function operates on copies of the input dictionaries `histories`
        and `meta_info` to avoid modifying the originals.
    """
    # --- Work on copies to avoid modifying original dictionaries ---
    histories_copy = histories.copy()
    meta_info_copy = meta_info.copy()
    # --- End copies ---

    plt.figure(figsize=(10, 6))

    # --- DYNAMIC BASELINE IDENTIFICATION & SELECTION/REMOVAL ---
    baseline_exp_key = None
    baseline_criteria_real = 5000
    baseline_criteria_fake = 0
    potential_baselines = []

    # Iterate over the copied meta_info to find potential baselines
    for exp_key, meta in meta_info_copy.items():
        if exp_key not in histories_copy: # Check history existence in the copy
            continue
        try:
            n_real = int(meta.get('n_real', -1))
            n_fake = int(meta.get('n_fake', -1))

            if n_real == baseline_criteria_real and n_fake == baseline_criteria_fake:
                potential_baselines.append(exp_key)

        except (ValueError, TypeError) as e:
            warnings.warn(f"Could not parse n_real/n_fake for experiment '{exp_key}'. Skipping for baseline check. Error: {e}")
            continue

    if len(potential_baselines) >= 1:
        # Select the first one found as the designated baseline
        baseline_exp_key = potential_baselines[0]
        print(f"Identified baseline experiment: '{baseline_exp_key}' (n_real={baseline_criteria_real}, n_fake={baseline_criteria_fake})")

        if len(potential_baselines) > 1:
            warnings.warn(f"Multiple experiments match baseline criteria: {potential_baselines}. Using '{baseline_exp_key}' and removing others from this plot.")
            # Remove the other baseline experiments from the copies
            for key_to_remove in potential_baselines:
                if key_to_remove != baseline_exp_key:
                    histories_copy.pop(key_to_remove, None) # Remove from histories copy
                    meta_info_copy.pop(key_to_remove, None)  # Remove from meta_info copy
                    print(f"--> Removed duplicate baseline '{key_to_remove}' from consideration for this plot.")

    else: # No baseline found
        print(f"Warning: No baseline experiment found matching criteria (n_real={baseline_criteria_real}, n_fake={baseline_criteria_fake}). Baseline will not be plotted.")
    # --- END DYNAMIC BASELINE IDENTIFICATION & SELECTION/REMOVAL ---


    # --- Initialization for Best/Worst Tracking (using copies) ---
    best_run_key = None
    best_run_hist = None
    worst_run_key = None
    worst_run_hist = None
    lower_is_better = 'loss' in METRIC.lower()

    if lower_is_better:
        best_val = float('inf')
        worst_val = float('-inf')
    else:
        best_val = float('-inf')
        worst_val = float('inf')

    all_vals = []

    # --- Main loop iterating through the potentially modified COPIES ---
    for exp, hist in histories_copy.items():

        # Skip the single designated baseline
        if exp == baseline_exp_key:
            continue

        # --- METRIC and META_INFO Checks (using copies) ---
        if METRIC not in hist:
            print(f"Warning: Metric '{METRIC}' not found in history for experiment '{exp}'. Skipping.")
            continue
        if not hist[METRIC]:
            print(f"Warning: Metric list for '{METRIC}' is empty for experiment '{exp}'. Skipping.")
            continue
        if exp not in meta_info_copy:
             print(f"Warning: Meta info not found for experiment '{exp}' in copy. Skipping plot details.")
             continue

        required_meta_keys = ['dataset', 'n_gen', 'used_gen', 'n_real', 'n_fake']
        current_meta = meta_info_copy[exp] # Use the copied meta info
        if not all(k in current_meta for k in required_meta_keys):
             print(f"Warning: Missing meta keys for experiment '{exp}'. Using defaults.")
             dataset, n_gen, used_gen, n_real, n_fake = "N/A", "N/A", "N/A", "N/A", "N/A"
        else:
            # Access values safely, perhaps using .get if structure isn't guaranteed
            dataset = current_meta.get('dataset', 'N/A')
            n_gen = current_meta.get('n_gen', 'N/A')
            used_gen = current_meta.get('used_gen', 'N/A')
            n_real = current_meta.get('n_real', 'N/A')
            n_fake = current_meta.get('n_fake', 'N/A')
        # --- End METRIC and META_INFO Checks ---

        # Plot individual run
        try:
            n_gen_int = int(n_gen)
            used_gen_int = int(used_gen)
            color_val = np.clip((used_gen_int + 1) / n_gen_int, 0, 1) if n_gen_int > 0 else 0.5
            run_color = (0.1, 0.2, color_val * 0.8 + 0.2, 0.25)

            col = (0.6, 0.6, 0.6, 0.3)
            plt.plot(hist[METRIC], color=col)
        except (ValueError, TypeError) as e:
            print(f"Warning: Could not parse gen info '{used_gen}', '{n_gen}' for run {exp}. Using default color. Error: {e}")
            plt.plot(hist[METRIC], color=col) # (0.5, 0.5, 0.5, 0.25))
        except Exception as e:
             print(f"Warning: An unexpected error occurred plotting run {exp}: {e}. Using default color.")
             plt.plot(hist[METRIC], color=col) # (0.5, 0.5, 0.5, 0.25))


        all_vals.append(hist[METRIC])
        current_final_val = hist[METRIC][-1]

        # Update Best/Worst Logic
        if lower_is_better:
            if current_final_val < best_val: best_val, best_run_key, best_run_hist = current_final_val, exp, hist[METRIC]
            if current_final_val > worst_val: worst_val, worst_run_key, worst_run_hist = current_final_val, exp, hist[METRIC]
        else:
            if current_final_val > best_val: best_val, best_run_key, best_run_hist = current_final_val, exp, hist[METRIC]
            if current_final_val < worst_val: worst_val, worst_run_key, worst_run_hist = current_final_val, exp, hist[METRIC]
    # --- End main loop ---

    if not all_vals:
        print(f"Error: No valid non-baseline data found for metric '{METRIC}' after filtering. Cannot generate plot.")
        # Plot only baseline if it exists? Or just return?
        if baseline_exp_key and baseline_exp_key in histories_copy and METRIC in histories_copy[baseline_exp_key] and histories_copy[baseline_exp_key][METRIC]:
             print("Plotting only the baseline run as no other runs were found/valid.")
             # Fall through to baseline plotting logic below
        else:
             plt.close() # Close the empty figure
             return


    # --- Padding and Aggregation (using all_vals from non-baseline runs) ---
    try:
        valid_vals = [v for v in all_vals if hasattr(v, '__iter__') and all(isinstance(x, (int, float)) for x in v)]
        if not valid_vals:
            # If all_vals was populated but contained non-numeric data
             print(f"Error: No valid numerical lists found in non-baseline runs for metric '{METRIC}'. Cannot calculate average/median.")
             avg, med = None, None # Signal that avg/med cannot be plotted
        else:
            max_len = max(map(len, valid_vals))
            all_vals_padded = [np.pad(np.array(a, dtype=float), (0, max_len - len(a)), mode='constant', constant_values=np.nan) for a in valid_vals]
            avg = np.nanmean(all_vals_padded, axis=0)
            med = np.nanmedian(all_vals_padded, axis=0)
    except ValueError as e:
        print(f"Error during padding or aggregation: {e}. Check data types in metric lists.")
        avg, med = None, None # Signal that avg/med cannot be plotted
    # --- End Padding and Aggregation ---


    # CUD color palette
    color_worst = '#D55E00' # Rust
    color_best = '#009E73'  # Teal
    color_avg = '#0072B2'  # Blue
    color_med = '#E69F00'  # Golden
    color_base = '#000000' # Black

    # Plot worst run if found (using meta_info_copy)
    if worst_run_key and worst_run_hist:
        meta_w = meta_info_copy.get(worst_run_key, {})
        used_gen_w = meta_w.get('used_gen', 'N/A')
        n_real_w = meta_w.get('n_real', 'N/A')
        n_fake_w = meta_w.get('n_fake', 'N/A')
        worst_label = f"Worst ({'Min' if lower_is_better else 'Max'} Final {METRIC}), n-gen: {used_gen_w}, gen: {n_fake_w} real: {n_real_w}"
        plt.plot(worst_run_hist, color=color_worst, linewidth=2, label=worst_label)
    elif len(all_vals) > 0: # Only warn if there *were* runs to compare
         print("Warning: Could not determine the worst run among non-baseline experiments.")

    # Plot best run if found (using meta_info_copy)
    if best_run_key and best_run_hist:
        meta_b = meta_info_copy.get(best_run_key, {})
        used_gen_b = meta_b.get('used_gen', 'N/A')
        n_real_b = meta_b.get('n_real', 'N/A')
        n_fake_b = meta_b.get('n_fake', 'N/A')
        best_label = f"Best ({'Max' if lower_is_better else 'Min'} Final {METRIC}), n-gen: {used_gen_b}, gen: {n_fake_b}, real: {n_real_b}"
        plt.plot(best_run_hist, color=color_best, linewidth=2, label=best_label)
    elif len(all_vals) > 0: # Only warn if there *were* runs to compare
        print("Warning: Could not determine the best run among non-baseline experiments.")

    # Plot average if calculable
    if avg is not None:
        plt.plot(avg, color=color_avg, linewidth=2, label="Average (excl. baseline)")

    # Plot median if calculable
    if med is not None:
        plt.plot(med, color=color_med, linewidth=2, linestyle='--', label="Median (excl. baseline)")

    # Plot the single designated baseline (using copies)
    if baseline_exp_key and baseline_exp_key in histories_copy and METRIC in histories_copy[baseline_exp_key] and histories_copy[baseline_exp_key][METRIC]:
        plt.plot(histories_copy[baseline_exp_key][METRIC], color=color_base, linewidth=2.5, linestyle=':', label=f'Baseline (R={baseline_criteria_real}, F={baseline_criteria_fake})')
    # No need for elif here, warning for missing baseline was handled during identification

    # --- Dynamic Title (using copies) ---
    plot_dataset_name = "Unknown Dataset"
    try:
        first_valid_key = next((k for k in histories_copy if k != baseline_exp_key and k in meta_info_copy and 'dataset' in meta_info_copy[k]), None)
        if first_valid_key:
            plot_dataset_name = meta_info_copy[first_valid_key]['dataset']
        elif baseline_exp_key and baseline_exp_key in meta_info_copy and 'dataset' in meta_info_copy[baseline_exp_key]:
             plot_dataset_name = meta_info_copy[baseline_exp_key]['dataset']
    except Exception as e:
        print(f"Could not reliably determine dataset name for title: {e}")

    plt.title(f"{title_prefix} {METRIC.replace('_', ' ').title()} - Dataset: {plot_dataset_name}, {title_suffix}, N-Gen: {target_gen}")
    # --- END Dynamic Title ---

    plt.xlabel("Epoch")
    plt.ylabel(METRIC.replace('_', ' ').title())
    # Only add legend if there are labeled lines
    handles, labels = plt.gca().get_legend_handles_labels()
    if labels:
         plt.legend(fontsize='small', loc='best')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()

    # Save or show plot
    if show:
        print("Displaying plot...")
        plt.show()
    else:
        try:
            save_path.parent.mkdir(parents=True, exist_ok=True)
            plt.savefig(save_path, bbox_inches='tight')
            print(f"Plot saved to {save_path}")
        except Exception as e:
            print(f"Error saving plot to {save_path}: {e}")

    plt.close() # Close the figure

### NOTE: 
there are only experiments with 3, 5, 7, 10 generators 

In [5]:
# load all histories: 

experiments_by_used_gen = {}

histories = {}
meta_info = {}
folders =[
    'MNIST_STRATIFIED_CLASSIFIERS_MADGAN_NEW',
    'MNIST_STRATIFIED_CLASSIFIERS_cMADGAN_NEW',
    'FASHION_STRATIFIED_CLASSIFIERS_MADGAN_NEW',
    'FASHION_STRATIFIED_CLASSIFIERS_cMADGAN_NEW'
]
suffix_map = {
    'MNIST_STRATIFIED_CLASSIFIERS_MADGAN_NEW':"MADGAN",
    'MNIST_STRATIFIED_CLASSIFIERS_cMADGAN_NEW':"cMADGAN",
    'FASHION_STRATIFIED_CLASSIFIERS_MADGAN_NEW':"MADGAN",
    'FASHION_STRATIFIED_CLASSIFIERS_cMADGAN_NEW':"cMADGAN"
}

for FOLDER in folders:
    DATASET, _, _, TYPE, _ = FOLDER.split('_')

    metrics = ["val_f1_score", "val_loss"]   
    base_path = Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/notebooks")
    strat_exp_path = base_path / 'experiments' / FOLDER 
    experiments = os.listdir(strat_exp_path)

    expansen_experiments = []
    replacement_experiments = []

    for exp in experiments: 
        if "real_5000_gen_0" in exp: 
            expansen_experiments.append(exp)
            replacement_experiments.append(exp)
        elif "real_5000" in exp: 
            expansen_experiments.append(exp)
        else:
            try:
                if int(exp.split("_")[-3]) < 5000:
                    replacement_experiments.append(exp)
            except Exception:
                pass

    for METRIC in metrics:
        print(f"CURRENT METRIC: {METRIC}")
        for target_gen in [3, 5, 7, 10]:
            print(f"CURRENT GENERATOR: {target_gen}")

            for exp in expansen_experiments: 
                meta = extract_info_from_experiment_name(exp)
                if meta['n_gen'] == target_gen:
                    try:
                        history = np.load(Path(strat_exp_path) / exp / 'training_history.npy', allow_pickle=True).item()
                        histories[exp] = history
                        meta_info[exp] = meta
                    except Exception:
                        pass
            
            histories = sort_dict_based_on_n_real_images(histories)
            meta_info = sort_dict_based_on_n_real_images(meta_info)

            if histories:
                plot_history_strat_classifiers(
                    histories, 
                    meta_info, 
                    Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/latex/master_thesis/abb/strat_classifier_performance") / FOLDER / "expansion_experiments" / f"{METRIC}_{TYPE}_{DATASET}_n_gen_{target_gen}_all.png",    
                    target_gen,
                    METRIC, 
                    title_prefix="Expansion Exp.:",
                    title_suffix=f"Type: {suffix_map[FOLDER]}"
                )
            histories = {}
            meta_info = {}

            for exp in replacement_experiments: 
                meta = extract_info_from_experiment_name(exp)
                if meta['n_gen'] == target_gen:
                    try:
                        history = np.load(Path(strat_exp_path) / exp / 'training_history.npy', allow_pickle=True).item()
                        histories[exp] = history
                        meta_info[exp] = meta
                    except Exception:
                        pass

            histories = sort_dict_based_on_n_real_images(histories)
            meta_info = sort_dict_based_on_n_real_images(meta_info)

            if histories:
                plot_history_strat_classifiers(
                    histories, 
                    meta_info, 
                    Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/latex/master_thesis/abb/strat_classifier_performance") / FOLDER / "replacement_experiments" / f"{METRIC}_{TYPE}_{DATASET}_n_gen_{target_gen}_all.png",    
                    target_gen,
                    METRIC,
                    title_prefix="Replacement Exp.:", 
                    title_suffix=f"Type: {suffix_map[FOLDER]}"
                )
            histories = {}
            meta_info = {}

CURRENT METRIC: val_f1_score
CURRENT GENERATOR: 3
Identified baseline experiment: '2025-04-21_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_0__trained_generators_3__images_real_5000_gen_0' (n_real=5000, n_fake=0)
--> Removed duplicate baseline '2025-04-22_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_1__trained_generators_3__images_real_5000_gen_0' from consideration for this plot.
--> Removed duplicate baseline '2025-04-22_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_2__trained_generators_3__images_real_5000_gen_0' from consideration for this plot.


C:\Users\NiXoN\AppData\Local\Temp\ipykernel_9928\3550676001.py:61: UserWarning: Multiple experiments match baseline criteria: ['2025-04-21_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_0__trained_generators_3__images_real_5000_gen_0', '2025-04-22_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_1__trained_generators_3__images_real_5000_gen_0', '2025-04-22_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_2__trained_generators_3__images_real_5000_gen_0']. Using '2025-04-21_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_0__trained_generators_3__images_real_5000_gen_0' and removing others from this plot.
  warnings.warn(f"Multiple experiments match baseline criteria: {potential_baselines}. Using '{baseline_exp_key}' and removing others from this plot.")


Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\latex\master_thesis\abb\strat_classifier_performance\MNIST_STRATIFIED_CLASSIFIERS_MADGAN_NEW\expansion_experiments\val_f1_score_MADGAN_MNIST_n_gen_3_all.png
Identified baseline experiment: '2025-04-21_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_0__trained_generators_3__images_real_5000_gen_0' (n_real=5000, n_fake=0)
--> Removed duplicate baseline '2025-04-22_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_1__trained_generators_3__images_real_5000_gen_0' from consideration for this plot.
--> Removed duplicate baseline '2025-04-22_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_2__trained_generators_3__images_real_5000_gen_0' from consideration for this plot.
Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\latex\master_thesis\abb\strat_classifier_performance\MNIST_STRATIFIED_CLASSIFIERS_MADGAN_NEW\replacement_experiments\val_f1_score_MADGAN_MNIST_n_gen_3_all.png
CURREN

C:\Users\NiXoN\AppData\Local\Temp\ipykernel_9928\3550676001.py:61: UserWarning: Multiple experiments match baseline criteria: ['2025-04-21_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_0__trained_generators_5__images_real_5000_gen_0', '2025-04-22_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_1__trained_generators_5__images_real_5000_gen_0', '2025-04-22_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_2__trained_generators_5__images_real_5000_gen_0', '2025-04-23_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_3__trained_generators_5__images_real_5000_gen_0', '2025-04-23_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_4__trained_generators_5__images_real_5000_gen_0']. Using '2025-04-21_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_0__trained_generators_5__images_real_5000_gen_0' and removing others from this plot.
  warnings.warn(f"Multiple experiments match baseline criteria: {potential_baseli

Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\latex\master_thesis\abb\strat_classifier_performance\MNIST_STRATIFIED_CLASSIFIERS_MADGAN_NEW\expansion_experiments\val_f1_score_MADGAN_MNIST_n_gen_5_all.png
Identified baseline experiment: '2025-04-21_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_0__trained_generators_5__images_real_5000_gen_0' (n_real=5000, n_fake=0)
--> Removed duplicate baseline '2025-04-22_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_1__trained_generators_5__images_real_5000_gen_0' from consideration for this plot.
--> Removed duplicate baseline '2025-04-22_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_2__trained_generators_5__images_real_5000_gen_0' from consideration for this plot.
--> Removed duplicate baseline '2025-04-23_Stratified_classifierExperiment_MNIST___MADGAN__used_generator_3__trained_generators_5__images_real_5000_gen_0' from consideration for this plot.
--> Removed duplicate baseline '2025

...

Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\latex\master_thesis\abb\strat_classifier_performance\FASHION_STRATIFIED_CLASSIFIERS_cMADGAN_NEW\expansion_experiments\val_f1_score_cMADGAN_FASHION_n_gen_5_all.png
Identified baseline experiment: '2025-04-21_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_0__trained_generators_5__images_real_5000_gen_0' (n_real=5000, n_fake=0)
--> Removed duplicate baseline '2025-04-22_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_1__trained_generators_5__images_real_5000_gen_0' from consideration for this plot.
--> Removed duplicate baseline '2025-04-22_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_2__trained_generators_5__images_real_5000_gen_0' from consideration for this plot.
--> Removed duplicate baseline '2025-04-23_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_3__trained_generators_5__images_real_5000_gen_0' from consideration for this plot.
--> Removed duplic

C:\Users\NiXoN\AppData\Local\Temp\ipykernel_9928\3550676001.py:61: UserWarning: Multiple experiments match baseline criteria: ['2025-04-21_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_0__trained_generators_7__images_real_5000_gen_0', '2025-04-22_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_1__trained_generators_7__images_real_5000_gen_0', '2025-04-22_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_2__trained_generators_7__images_real_5000_gen_0', '2025-04-23_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_3__trained_generators_7__images_real_5000_gen_0', '2025-04-25_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_4__trained_generators_7__images_real_5000_gen_0', '2025-04-25_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_5__trained_generators_7__images_real_5000_gen_0', '2025-04-25_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_6__trained_generators_7__im

Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\latex\master_thesis\abb\strat_classifier_performance\FASHION_STRATIFIED_CLASSIFIERS_cMADGAN_NEW\expansion_experiments\val_f1_score_cMADGAN_FASHION_n_gen_7_all.png
Identified baseline experiment: '2025-04-21_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_0__trained_generators_7__images_real_5000_gen_0' (n_real=5000, n_fake=0)
--> Removed duplicate baseline '2025-04-22_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_1__trained_generators_7__images_real_5000_gen_0' from consideration for this plot.
--> Removed duplicate baseline '2025-04-22_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_2__trained_generators_7__images_real_5000_gen_0' from consideration for this plot.
--> Removed duplicate baseline '2025-04-23_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_3__trained_generators_7__images_real_5000_gen_0' from consideration for this plot.
--> Removed duplic

C:\Users\NiXoN\AppData\Local\Temp\ipykernel_9928\3550676001.py:61: UserWarning: Multiple experiments match baseline criteria: ['2025-04-21_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_0__trained_generators_10__images_real_5000_gen_0', '2025-04-22_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_1__trained_generators_10__images_real_5000_gen_0', '2025-04-22_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_2__trained_generators_10__images_real_5000_gen_0', '2025-04-23_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_3__trained_generators_10__images_real_5000_gen_0', '2025-04-23_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_4__trained_generators_10__images_real_5000_gen_0', '2025-04-23_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_6__trained_generators_10__images_real_5000_gen_0', '2025-04-23_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_7__trained_generators

Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\latex\master_thesis\abb\strat_classifier_performance\FASHION_STRATIFIED_CLASSIFIERS_cMADGAN_NEW\expansion_experiments\val_f1_score_cMADGAN_FASHION_n_gen_10_all.png
Identified baseline experiment: '2025-04-21_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_0__trained_generators_10__images_real_5000_gen_0' (n_real=5000, n_fake=0)
--> Removed duplicate baseline '2025-04-22_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_1__trained_generators_10__images_real_5000_gen_0' from consideration for this plot.
--> Removed duplicate baseline '2025-04-22_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_2__trained_generators_10__images_real_5000_gen_0' from consideration for this plot.
--> Removed duplicate baseline '2025-04-23_Stratified_classifierExperiment_FASHION___cMADGAN__used_generator_3__trained_generators_10__images_real_5000_gen_0' from consideration for this plot.
--> Removed d

In [6]:
def plot_history_simple(
    histories: dict, meta_info: dict, save_path: Path, metric, title_prefix, title_suffix, show: bool = False
) -> None:
    """
    Simplified version of plotting metric history with baseline, best, worst, average, and median.
    Assumes no multiple-generator logic is needed.

    Args:
        histories (dict): Dict of history objects.
        meta_info (dict): Dict with 'n_real', 'n_fake', and 'dataset'.
        save_path (Path): Path to save plot.q
        metric (str): Metric to plot.
        title_prefix (str): Prefix for the plot title.
        title_suffix (str): Suffix for the plot title.
        show (bool): If True, display plot.
    """
    histories_copy = histories.copy()
    meta_info_copy = meta_info.copy()

    plt.figure(figsize=(10, 6))

    # Identify baseline
    baseline_key = None
    baseline_criteria_real, baseline_criteria_fake = 5000, 0
    for k, v in meta_info_copy.items():
        if v.get('n_real') == baseline_criteria_real and v.get('n_fake') == baseline_criteria_fake:
            baseline_key = k
            break

    all_vals = []
    best_val = float('-inf')
    worst_val = float('inf')
    best_key, worst_key = None, None
    best_hist, worst_hist = None, None
    lower_is_better = 'loss' in metric.lower()

    for k, hist in histories_copy.items():
        if metric not in hist or not hist[metric]:
            continue
        vals = hist[metric]
        if k == baseline_key:
            continue  # skip plotting for now; baseline handled later
        final_val = vals[-1]
        all_vals.append(vals)
        if (lower_is_better and final_val < best_val) or (not lower_is_better and final_val > best_val):
            best_val, best_key, best_hist = final_val, k, vals
        if (lower_is_better and final_val > worst_val) or (not lower_is_better and final_val < worst_val):
            worst_val, worst_key, worst_hist = final_val, k, vals
        plt.plot(vals, color=(0.6, 0.6, 0.6, 0.3))

    # Compute avg/median (excluding baseline)
    try:
        max_len = max(len(v) for v in all_vals)
        padded = [np.pad(v, (0, max_len - len(v)), constant_values=np.nan) for v in all_vals]
        avg = np.nanmean(padded, axis=0)
        med = np.nanmedian(padded, axis=0)
    except:
        avg, med = None, None

    # CUD color palette
    color_worst = '#D55E00' # Rust
    color_best = '#009E73'  # Teal
    color_avg = '#0072B2'  # Blue
    color_med = '#E69F00'  # Golden
    color_base = '#000000' # Black

    # Plot special curves
    if baseline_key and metric in histories_copy.get(baseline_key, {}):
        plt.plot(histories_copy[baseline_key][metric], color='black', linestyle=':', linewidth=2.5,
                 label=f'Baseline (R={baseline_criteria_real}, F={baseline_criteria_fake})')
    if best_hist:
        plt.plot(best_hist, color=color_best, linewidth=2, label=f'Best Run, gen: {best_key.split("_")[-1]}, real: {best_key.split("_")[-3]}')
    if worst_hist:
        plt.plot(worst_hist, color=color_worst, linewidth=2, label=f'Worst Run, gen: {worst_key.split("_")[-1]}, real: {worst_key.split("_")[-3]}')
    if avg is not None:
        plt.plot(avg, color=color_avg, linewidth=2, label='Average (excl. baseline)')
    if med is not None:
        plt.plot(med, color=color_med, linewidth=2, linestyle='--', label='Median (excl. baseline)')

    # Title and labels
    dataset = meta_info_copy.get(baseline_key, {}).get('dataset', 'Unknown') if baseline_key else 'Unknown'
    plt.title(f"{title_prefix} {metric.replace('_', ' ').title()} - Dataset: {dataset}, {title_suffix}")
    plt.xlabel("Epoch")
    plt.ylabel(metric.replace('_', ' ').title())
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(fontsize='small', loc='best')
    plt.tight_layout()

    if show:
        plt.show()
    else:
        try:
            save_path.parent.mkdir(parents=True, exist_ok=True)
            plt.savefig(save_path, bbox_inches='tight')
            print(f"Plot saved to {save_path}")
        except Exception as e:
            print(f"Error saving plot: {e}")

    plt.close()


In [7]:
metrics = ["val_accuracy", "val_f1_score", "val_loss"]   

base_path = Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/notebooks")
strat_exp_path = base_path / 'experiments'  
suffix_map = {
    'FASHION_STRATIFIED_CLASSIFIERS_COND_GAN':"Conditional",
    'FASHION_STRATIFIED_CLASSIFIERS_VANILLA_GAN':"Deep Conv.",
    'MNIST_STRATIFIED_CLASSIFIERS_COND_GAN':"Conditional",
    'MNIST_STRATIFIED_CLASSIFIERS_VANILLA_GAN':"Deep Conv."
}

for folder in [
    "FASHION_STRATIFIED_CLASSIFIERS_COND_GAN",
    "FASHION_STRATIFIED_CLASSIFIERS_VANILLA_GAN",
    "MNIST_STRATIFIED_CLASSIFIERS_COND_GAN",
    "MNIST_STRATIFIED_CLASSIFIERS_VANILLA_GAN",
]: 
    strat_exp_path = base_path / 'experiments' / folder 
    experiments = os.listdir(strat_exp_path)
    histories = {}
    meta_info = {}

    DATASET = folder.split("_")[0]

    expansen_experiments = []
    replacement_experiments = []
        
    for exp in experiments: 
        if "real_5000_gen_0" in exp: 
            expansen_experiments.append(exp)
            replacement_experiments.append(exp)
        elif "real_5000" in exp: 
            expansen_experiments.append(exp)
        else:
            try:
                if int(exp.split("_")[-3]) < 5000:
                    replacement_experiments.append(exp)
            except Exception:
                pass


    for METRIC in metrics:
        print(f"CURRENT METRIC: {METRIC}")

        histories = {}
        meta_info = {}
        
        for exp in expansen_experiments: 
            meta = extract_info_from_experiment_name(exp)
            try:
                history = np.load(Path(strat_exp_path) / exp / 'training_history.npy', allow_pickle=True).item()
                histories[exp] = history
                meta_info[exp] = meta
            except Exception as e:
                print("1111111111111ERROROROEROEOREOROEROE")
                print( e)
                pass
        histories = sort_dict_based_on_n_real_images(histories)
        meta_info = sort_dict_based_on_n_real_images(meta_info)

        plot_history_simple(
            histories, meta_info, metric=METRIC, 
            title_prefix="Expansion Exp.:", 
            title_suffix=f"Type: {suffix_map[folder]}",
            save_path= Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/latex/master_thesis/abb/strat_classifier_performance") / folder / "expansion_experiments" / f"{METRIC}_{folder.split('_')[-2:-1]}_{DATASET}_all.png",    
        )

        histories = {}
        meta_info = {}
        
        for exp in replacement_experiments: 
            meta = extract_info_from_experiment_name(exp)
            try: 
                history = np.load(Path(strat_exp_path) / exp / 'training_history.npy', allow_pickle=True).item()
                histories[exp] = history
                meta_info[exp] = meta
            except Exception as e:
                print("2222222222222ERROROROEROEOREOROEROE")
                print( e)
                pass
        histories = sort_dict_based_on_n_real_images(histories)

        
        meta_info = sort_dict_based_on_n_real_images(meta_info)

        plot_history_simple(
            histories, meta_info, metric=METRIC, 
            title_prefix="Replacement Exp.:", 
            title_suffix=f"Type: {suffix_map[folder]}",
            save_path= Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/latex/master_thesis/abb/strat_classifier_performance") / folder / "replacement_experiments" / f"{METRIC}_{folder.split('_')[-2:-1]}_{DATASET}_all.png",    
        )        

CURRENT METRIC: val_accuracy
Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\latex\master_thesis\abb\strat_classifier_performance\FASHION_STRATIFIED_CLASSIFIERS_COND_GAN\expansion_experiments\val_accuracy_['COND']_FASHION_all.png
Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\latex\master_thesis\abb\strat_classifier_performance\FASHION_STRATIFIED_CLASSIFIERS_COND_GAN\replacement_experiments\val_accuracy_['COND']_FASHION_all.png
CURRENT METRIC: val_f1_score
Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\latex\master_thesis\abb\strat_classifier_performance\FASHION_STRATIFIED_CLASSIFIERS_COND_GAN\expansion_experiments\val_f1_score_['COND']_FASHION_all.png
Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\latex\master_thesis\abb\strat_classifier_performance\FASHION_STRATIFIED_CLASSIFIERS_COND_GAN\replacement_experiments\val_f1_score_['COND']_FASHION_all.png
CURRENT METRIC: val_loss
Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\l

In [8]:
metrics = ["val_accuracy", "val_f1_score", "val_loss"]   

base_path = Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/notebooks")
strat_exp_path = base_path / 'experiments/TDA_STRATIFIED_CLASSIFIER'  

for folder in [
    "tda_fashion_mnist", 
    "tda_mnist"
]: 
    strat_exp_path = base_path / 'experiments/TDA_STRATIFIED_CLASSIFIERS' / folder 
    experiments = os.listdir(strat_exp_path)
    histories = {}
    meta_info = {}

    expansen_experiments = []
    replacement_experiments = []

    DATASET = folder.split("_")[1]
        
    for exp in experiments: 
        if "real_5000_gen_0" in exp: 
            expansen_experiments.append(exp)
            replacement_experiments.append(exp)
        elif "real_5000" in exp: 
            expansen_experiments.append(exp)
        else:
            try:
                if int(exp.split("_")[-3]) < 5000:
                    replacement_experiments.append(exp)
            except Exception:
                pass


    for METRIC in metrics:
        print(f"CURRENT METRIC: {METRIC}")    
        histories = {}
        meta_info = {}
        
        for exp in expansen_experiments: 
            meta = extract_info_from_experiment_name(exp)
            try:
                history = np.load(Path(strat_exp_path) / exp / 'training_history.npy', allow_pickle=True).item()
                histories[exp] = history
                meta_info[exp] = meta
            except Exception as e:
                print("1111111111111ERROROROEROEOREOROEROE")
                print( e)
                pass
        histories = sort_dict_based_on_n_real_images(histories)
        meta_info = sort_dict_based_on_n_real_images(meta_info)    
        plot_history_simple(
            histories, meta_info, metric=METRIC, 
            title_prefix="Expansion Exp.:", 
            title_suffix=f"Type: TDA",
            save_path= Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/latex/master_thesis/abb/strat_classifier_performance") / folder / "expansion_experiments" / f"{METRIC}_{folder}_{DATASET}_all.png",    
        )    
        histories = {}
        meta_info = {}
        
        for exp in replacement_experiments: 
            meta = extract_info_from_experiment_name(exp)
            try: 
                history = np.load(Path(strat_exp_path) / exp / 'training_history.npy', allow_pickle=True).item()
                histories[exp] = history
                meta_info[exp] = meta
            except Exception as e:
                print("2222222222222ERROROROEROEOREOROEROE")
                print( e)
                pass
        histories = sort_dict_based_on_n_real_images(histories) 
        
        meta_info = sort_dict_based_on_n_real_images(meta_info)    
        plot_history_simple(
            histories, meta_info, metric=METRIC, 
            title_prefix="Replacement Exp.:", 
            title_suffix=f"Type: TDA",
            save_path= Path("C:/Users/NiXoN/Desktop/_thesis/mad_gan_thesis/latex/master_thesis/abb/strat_classifier_performance") / folder / "replacement_experiments" / f"{METRIC}_{folder}_{DATASET}_all.png", 
        )        

CURRENT METRIC: val_accuracy
Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\latex\master_thesis\abb\strat_classifier_performance\tda_fashion_mnist\expansion_experiments\val_accuracy_tda_fashion_mnist_fashion_all.png
Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\latex\master_thesis\abb\strat_classifier_performance\tda_fashion_mnist\replacement_experiments\val_accuracy_tda_fashion_mnist_fashion_all.png
CURRENT METRIC: val_f1_score
Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\latex\master_thesis\abb\strat_classifier_performance\tda_fashion_mnist\expansion_experiments\val_f1_score_tda_fashion_mnist_fashion_all.png
Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\latex\master_thesis\abb\strat_classifier_performance\tda_fashion_mnist\replacement_experiments\val_f1_score_tda_fashion_mnist_fashion_all.png
CURRENT METRIC: val_loss
Plot saved to C:\Users\NiXoN\Desktop\_thesis\mad_gan_thesis\latex\master_thesis\abb\strat_classifier_performance\